In [1]:
#Script to analyze data from RCP-PCR experiments.
#Run the script in the RCP-PCR directory to analyze test data.
#To analyze own data, prepare a directory with name of data under RCP-PCR/Data with uncompressed R1/R2 fastq files.

In [2]:
import os 
import re
import sys
import numpy as np
import multiprocessing as mp
from Bio import SeqIO
import operator
from pprint import pprint
from tqdm import tqdm
import pickle

In [3]:
# Put directory name under 'Data' with fastq files.
dir_name    = 'test'

#Tag file
tag_file    = 'tag_assignment_test.csv'

# Number of threads used to execute commands.
threads     = 2

In [4]:
#Identify the fastq files to create small fasta files for easier handling.


fastq_files = []
PATH = os.path.abspath(".")
lis = os.listdir("%s/Data/%s" % (PATH,dir_name))
for f in lis:
    if f[-6:] == ".fastq" :
        fastq_files.append(f)
print("List of input fastq files: %s \n\n (There should be two files corresponding to R1/R2.)"%(fastq_files))


List of input fastq files: ['test_R1.fastq', 'test_R2.fastq'] 

 (There should be two files corresponding to R1/R2.)


In [5]:
#Creating directries for data proccessing.

try:
    os.makedirs('%s/Data/%s/fragmented_fasta'%(PATH,dir_name))
except OSError:
    pass

try:
    os.makedirs('%s/Data/%s/blast'%(PATH,dir_name))
    os.makedirs('%s/Data/%s/blast/sh.blast'%(PATH,dir_name))
    os.makedirs('%s/Data/%s/blast/out.blast'%(PATH,dir_name))
except OSError:
    pass

try:
    os.makedirs('%s/Data/%s/identification'%(PATH,dir_name))
    os.makedirs('%s/Data/%s/identification/sh.identification'%(PATH,dir_name))
    os.makedirs('%s/Data/%s/identification/out.identification'%(PATH,dir_name))
except OSError:
    pass

try:
    os.makedirs('%s/Data/%s/count'%(PATH,dir_name))
    os.makedirs('%s/Data/%s/count/sh.count'%(PATH,dir_name))
    os.makedirs('%s/Data/%s/count/out.count'%(PATH,dir_name))
except OSError:
    pass

In [6]:
#Defining function to create small fasta files.
def make_fasta_from_listoftups(L,name):
    with open("%s"%(name),"w") as f:
        for I in L:
            #print I
            f.write('>%s\n'%(str(I[0])))
            f.write('%s\n'%(str(I[1])))
        f.close()
        print("Made fna file : %s"%(name))

def make_fragmented_fasta(F,split_num,read_length,out_dir,dir_name):
    line_c  = 0
    read_c  = 0
    file_count   = 0
    with open(F,"r") as f:
        fasta_tup = []
        for line in f:
            if (dir_name =='test'):
                if line_c < (split_num*4):
                    if (line_c % 4 == 0):
                        read_ID = line.split(" ")[0].replace(":","_").replace("@","")

                    elif (line_c % 4 == 1):
                        if read_length > len(line):
                            seq = line
                        else:
                            seq     = line[:read_length]

                    elif (line_c % 4 == 3):
                        qscore  = line 
                        fasta_tup.append( (read_ID,seq) )
                        read_c +=1

                        if (read_c % split_num == 0):
                            file_count +=1

                            f_name = ("").join([F.split("/")[-1].split(".")[0] , "_%d"%(file_count),".fna"])
                            out    = ("/").join( [out_dir,f_name]) 

                            make_fasta_from_listoftups(fasta_tup, out )


                            fasta_tup = []
                    line_c += 1
            
            else:                
                if (line_c % 4 == 0):
                    read_ID = line.split(" ")[0].replace(":","_").replace("@","")

                elif (line_c % 4 == 1):
                    if read_length > len(line):
                        seq = line
                    else:
                        seq     = line[:read_length]

                elif (line_c % 4 == 3):
                    qscore  = line 
                    fasta_tup.append( (read_ID,seq) )
                    read_c +=1

                    if (read_c % split_num == 0):
                        file_count +=1

                        f_name = ("").join([F.split("/")[-1].split(".")[0] , "_%d"%(file_count),".fna"])
                        out    = ("/").join( [out_dir,f_name]) 

                        make_fasta_from_listoftups(fasta_tup, out )


                        fasta_tup = []
                line_c += 1


        f.close()
    pass

In [7]:
#Creating directory for fragmented fasta files and executing the function above. 
for F in fastq_files:
     make_fragmented_fasta("%s/Data/%s/%s"%(PATH,dir_name,F),4000000,250,"%s/Data/%s/fragmented_fasta"%(PATH,dir_name),dir_name) 

In [8]:
#Creating BLAST database.
os.system("makeblastdb -in %s/const-seq.fna -dbtype nucl"%(PATH))
db = '%s/const-seq.fna'%(PATH)

# You should be able to see the following output in your terminal 
#
#Building a new DB, current time: 03/09/2021 13:20:38
#New DB name:   /Users/username/GitHub/BFG-Y2H/RCP-PCR/const-seq.fna
#New DB title:  /Users/username/GitHub/BFG-Y2H/RCP-PCR/const-seq.fna
#Sequence type: Nucleotide
#Keep MBits: T
#Maximum file size: 1000000000B
#Adding sequences from FASTA; added 32 sequences in 0.00142884 seconds.

In [9]:
# Deefining multi-proccessing handling.
def subprocces_sh(core,sh_dir):
    command_L = get_sh(sh_dir)
    command_L = ["sh %s/%s"%(sh_dir,i) for i in command_L]
    if core > len(command_L):
        core = len(command_L)
    n = len(command_L)/core
    pool = mp.Pool(core)
    feed = {}
    for i in range(core):
        feed.update({i:[]})
    lp = len(command_L)
    ky = range(0,lp)
    v = 0
    for i in range(lp):
        feed[v].append(command_L[0])
        del command_L[0]
        v +=1
        if (v==core):
            v = 0
    fed = []
    for i in range(0,core):
        fed.append(feed[i])
    
    results = pool.map(run_sh,fed)
    pass

def get_sh(x):
    files = []
    lis = os.listdir("%s" %x)
    for i in lis:
        if i[-3:] == ".sh" :
            files.append(i)
    return files

def run_sh(sh_L):
    for sh in sh_L:
        #print("Running command : %s\n"%(sh))
        os.system(sh)
    
        


In [10]:
# Prepare .sh files for executing BLAST

#Listing fragmented fastas files.
fasta_files = os.listdir("%s/Data/%s/fragmented_fasta" % (PATH,dir_name)) 

#Generating sh files.
sh = 1
for f in fasta_files:
    command = "blastn -task blastn-short -strand plus -db %s/const-seq.fna -outfmt 10 -evalue 1e-3 -query %s/Data/%s/fragmented_fasta/%s -out %s/Data/%s/blast/out.blast/%s.blast"%(PATH,PATH,dir_name,f,PATH,dir_name,f.split(".")[0])
    f_name = "%s/Data/%s/blast/sh.blast/blast_%d.sh"%(PATH,dir_name,sh)
    with open(f_name,"w") as F:
        F.write(command)
        print("Generated %s"%(f_name))
    F.close()
    sh +=1


    

Generated /Users/danyamamotoevans/GitHub/BFG-Y2H/RCP-PCR/Data/test/blast/sh.blast/blast_1.sh
Generated /Users/danyamamotoevans/GitHub/BFG-Y2H/RCP-PCR/Data/test/blast/sh.blast/blast_2.sh


In [11]:
# Executing BLAST sh files
print("Dividing and executing BLAST.sh files on %d threads (This will take a while)\n"%(threads))
subprocces_sh(threads,"%s/Data/%s/blast/sh.blast"%(PATH,dir_name))
print ("Done")

Dividing and executing BLAST.sh files on 2 threads (This will take a while)

Done


In [12]:
#Reading reference files
bar2num_file = "%s/bar2num.txt"%(PATH)
bar_d = {}
with open(bar2num_file,"r") as F:
    for line in F:
        cols = line.split("\n")[0].split(",")
        num = int(cols[0])  
        bar = cols[1]
        bar_d[bar] = num
    F.close()

tag_f    = "%s/%s"%(PATH,tag_file)
tags = {}
with open(tag_f,"r") as F:
    for line in F:
        cols = line.split("\n")[0].split(",")
        #AD003,AD,BC,P01-P01
        plate        = cols[0]
        barcode_type = cols[1] 
        PCR_type     = cols[2] 
        tag          = cols[3] 
        tags[tag] = [plate,barcode_type,PCR_type,tag]
    F.close()    

In [13]:
#Function to parse fasta file to doctionary.
def fasta2dict(name):
    d = {}
    for record in SeqIO.parse(name,"fasta"):
        d.update({record.id :  str(record.seq)})
    return d


def remove_key(d, key):
    r = dict(d)
    del r[key]
    return r



#Function to parse BLAST output to dictionary
def parse_blast_output2dict(blast_R1,blast_R2,PATH,dir_name,tag_f): 
    # BLAST database sequences in dict format
    seq_DB   = fasta2dict("%s/const-seq.fna"%(PATH))
    
    
    blast_d = {}
    
    with open(blast_R1) as F:
        for line in F:
            cols           = line.split("\n")[0].split(",")
            #print(cols)
            read           = cols[0]   
            subject        = cols[1]  
            str_on_read    = int(cols[6])  
            end_on_read    = int(cols[7])
            str_on_subject = int(cols[8])
            end_on_subject = int(cols[9])
            evalue         = float(cols[10])
            
           
            if (str_on_read < end_on_read):
                if (str_on_subject ==1):
                    if (end_on_subject==len(seq_DB[subject])):
                        element = {"str_on_read":str_on_read,
                                   "end_on_read":end_on_read,
                                   "evalue":evalue}
                        try:
                            blast_d[read]["R1"][subject] = element
                        except KeyError:
                            try:
                                blast_d[read]["R1"]       = {}
                                blast_d[read]["R1"][subject] = element
                            except KeyError:
                                blast_d[read] = {}
                                blast_d[read]["R1"]       = {}
                                blast_d[read]["R1"][subject] = element
 
        F.close()
    #pprint(blast_d)  
    with open(blast_R2) as F:
        for line in F:
            cols             = line.split("\n")[0].split(",")
            read             = cols[0]  
            subject          = cols[1]   
            str_on_read      = int(cols[6]   )
            end_on_read      = int(cols[7]   )
            str_on_subject   = int(cols[8]   )
            end_on_subject   = int(cols[9]   )
            evalue           = float(cols[10])  
    
            if (str_on_read < end_on_read):
                if (str_on_subject ==1):
                    if (end_on_subject==len(seq_DB[subject])):
                        
                        element = {"str_on_read":str_on_read,
                                       "end_on_read":end_on_read,
                                       "evalue":evalue}
                        try:
                            blast_d[read]["R2"][subject] = element
                        except KeyError:
                            try:
                                blast_d[read]["R2"] = {}
                                blast_d[read]["R2"][subject] = element
                            except KeyError:
                                blast_d[read] = {}
                                blast_d[read]["R2"] = {}
                                blast_d[read]["R2"][subject] = element

        F.close()
    
        
    
    return blast_d         

In [14]:
# Function to assign RCP-PCR category and extract index sequences. 
def assign_category(read_d):

    R1 = read_d['R1']
    R2 = read_d['R2']
    read_R1_seq = read_d['R1']['seq']
    read_R2_seq = read_d['R2']['seq']
    

    # PS1.0 and PS2.0 at the reasonable positions? (end at <40bp)
    PS_pos = 0

    P_SEQ1 = 0
    P_SEQ2 = 0

    if( ('PS1.0-primer'in R1.keys()) and ('PS2.0-primer' in R2.keys()) ):
        if( (R1['PS1.0-primer']['end_on_read'] < 40) and (R2['PS2.0-primer']['end_on_read'] < 40) ):
            P_SEQ1 = read_R1_seq[R1['PS1.0-primer']['str_on_read']-10:R1['PS1.0-primer']['str_on_read']-1]
            P_SEQ2 = read_R2_seq[R2['PS2.0-primer']['str_on_read']-10:R2['PS2.0-primer']['str_on_read']-1]
            PS_pos = 1
            
    # Row and Column priming sites from the reasonable positions? (start at <50bp)
    # Which category? DB-BC / DB-lox / AD-BC / AD-lox?
    DB_BC = 0
    DB_lox = 0
    AD_BC = 0
    AD_lox = 0
    
    target=0
    # Specific category assignment?
    category = 0
    R_SEQ = 0
    C_SEQ = 0
    
    
    if(PS_pos ==1):
        R_seq = {}
        C_seq = {}
        
        #DB-BC
        if( ('DBU1-primer' in R1.keys()) and ('DBD2-primer' in R2.keys())):
            if (R1['PS1.0-primer']['end_on_read'] < R1['DBU1-primer']['str_on_read']):
               if (R1['DBU1-primer']['str_on_read'] < 50):
                   if (R2['PS2.0-primer']['end_on_read'] < R2['DBD2-primer']['str_on_read']):
                        if (R2['DBD2-primer']['str_on_read'] < 50):

                            R_seq['DB-BC'] = read_R1_seq[R1['PS1.0-primer']['end_on_read']:R1['DBU1-primer']['str_on_read']+1-2]
                            C_seq['DB-BC'] = read_R2_seq[R2['PS2.0-primer']['end_on_read']:R2['DBD2-primer']['str_on_read']+1-2] 
                            DB_BC = 1
                            target+=1

               
        #DB-lox
        if (('DBloxP-primer' in R1.keys()) and ('DBlox2272-primer' in R2.keys())):     
            if(R1['PS1.0-primer']['end_on_read'] < R1['DBloxP-primer']['str_on_read']):
               if( R1['DBloxP-primer']['str_on_read'] < 50 ):
                   if (R2['PS2.0-primer']['end_on_read']< R2['DBlox2272-primer']['str_on_read']):
                       if(R2['DBlox2272-primer']['str_on_read'] < 50):
               
                            R_seq['DB-lox'] = read_R1_seq[R1['PS1.0-primer']['end_on_read']:R1['DBloxP-primer']['str_on_read']+1-2]
                            C_seq['DB-lox'] = read_R2_seq[R2['PS2.0-primer']['end_on_read']:R2['DBlox2272-primer']['str_on_read']+1-2] 
                            DB_lox = 1
                            target+=1
        #AD-BC
        if( ('ADU1-primer' in R1.keys()) and ('ADD2-primer' in R2.keys()) ):
            if( (R1['PS1.0-primer']['end_on_read'] < R1['ADU1-primer']['str_on_read'])):
               if (R1['ADU1-primer']['str_on_read'] < 50):
                   if (R2['PS2.0-primer']['end_on_read'] < R2['ADD2-primer']['str_on_read']):
                        if (R2['ADD2-primer']['str_on_read'] < 50):

                            R_seq['AD-BC'] = read_R1_seq[R1['PS1.0-primer']['end_on_read']:R1['ADU1-primer']['str_on_read']+1-2]
                            C_seq['AD-BC'] = read_R2_seq[R2['PS2.0-primer']['end_on_read']:R2['ADD2-primer']['str_on_read']+1-2] 
                            AD_BC = 1
                            target+=1
               
        #AD-lox
        if ('ADloxP-primer' in R1.keys()) and ('ADlox2272-primer' in R2.keys())  :   
            if(R1['PS1.0-primer']['end_on_read'] < R1['ADloxP-primer']['str_on_read']):
               if ( R1['ADloxP-primer']['str_on_read'] < 50 ):
                   if (R2['PS2.0-primer']['end_on_read']< R2['ADlox2272-primer']['str_on_read']):
                       if (R2['ADlox2272-primer']['str_on_read'] < 50):
               
                            R_seq['AD-lox'] = read_R1_seq[R1['PS1.0-primer']['end_on_read']:R1['ADloxP-primer']['str_on_read']+1-2]
                            C_seq['AD-lox'] = read_R2_seq[R2['PS2.0-primer']['end_on_read']:R2['ADlox2272-primer']['str_on_read']+1-2] 
                            AD_lox = 1
                            target+=1

       

        if(target == 1):
            if(DB_BC==1):
                category = 'DB-BC'
            elif(DB_lox==1):
                category = 'DB-lox'
            elif(AD_BC==1):
                category = 'AD-BC'
            elif(AD_lox==1):
                category = 'AD-lox'
       
            R_SEQ = R_seq[category]
            C_SEQ = C_seq[category]
               
        
    return (target,category,P_SEQ1,P_SEQ2,R_SEQ,C_SEQ)



In [15]:
# Defining small functions for calling barcode category and tags index.

def rv_comp(seq):
    bases = {"A":"T","T":"A","G":"C","C":"G","N":"N","a":"t","t":"a","g":"c","c":"g","n":"n"}
    seq_l = list(seq)
    rv_comp = [bases[base] for base in seq_l]
    rv_comp.reverse()
    rv_comp
    return ("").join(rv_comp)


def BC_match(R1,R2):
    r1 = R1
    r2 = rv_comp(R2) 
    #print(r1)
    #print(r2)
    rel = {}
    max_match = 0

    for i in range(0,len(r1)):
        n1 = r1[i]
        for j in range(0,len(r2)):
            n2 = r2[j]
            if(n1 == n2):
                diff = j-i
                try:
                    rel[diff] +=1
                    if max_match < rel[diff]:
                            max_match       = rel[diff]
                            max_match_diff  = diff
                except KeyError:
                    rel[diff] = 1

    BC = []               
    
    if rel[max_match_diff] >= 18:
        diff  = max_match_diff
        count = rel[max_match_diff]
        r1_bc = r1
        r2_bc = r2[diff:]
        
        for i in range(0,len(r1)):    
            nuc_r1 = r1_bc[i]
            if (len(r2_bc)-1) > i:
                nuc_r2 = r2_bc[i]
                if nuc_r1 == nuc_r2:
                    n = nuc_r1
                elif((nuc_r1=="N") and (nuc_r2 != "N")):
                    n = nuc_r2
                elif((nuc_r1!="N") and (nuc_r2 == "N")):
                    n = nuc_r2
                else:
                    n = "N"
            else:
                n = nuc_r1
            
            BC.append(n)
    #print(("").join(BC))
    return ("").join(BC)

def tag_calling(bar2num,seq):
    if len(seq) < 12:
        
        target_hits = []
        for tag in bar2num:
            count = {}
            for k in range(0,len(tag)):
                k_nuc = tag[k]

                for i in range(0,len(seq)):
                    i_nuc = seq[i]

                    rel = k-i
                    if(k_nuc == i_nuc):
                        try:
                            count[rel] += 1
                        except KeyError:
                            count[rel] = 1
                    else:
                        try:
                            count[rel] += 0
                        except KeyError:
                            count[rel] = 0
            #pprint(count)  
            if len(count) > 0:
                max_value     = max(count.values()) 
                max_match_rel = [k for k, v in count.items() if v == max_value] # getting all keys containing the `maximum`
                for i in max_match_rel:
                    target_hits.append([tag,i,count[i]])
                #pprint(target_hits)

        target_hits.sort(key=lambda x: x[2],reverse=True)
        #pprint(target_hits)
        if (len(target_hits) >1):
            if(target_hits[0][2] == target_hits[1][2]):
                val = 0
            elif(target_hits[0][2]>5): 
                called_bar = bar2num[target_hits[0][0]]
                val = called_bar
            else:
                val = 0
        elif(len(target_hits)==1):
            #print (target_hits)
            if(target_hits[0][2]>5): 
                called_bar = bar2num[target_hits[0][0]]
                val = called_bar
        else:
            val = 0

    else:
        val = 0
    return val

In [16]:
# Definfing function for analysing status of each element within barcode cassette.
def analyze_stat(category,R1,R2,seq_DB):
    stat = {"R1":{},"R2":{}}
    #print (category)
    if (category == "DB-BC"):    
        # Setting default status 
        stat["R1"]['DBU1-primer']  = 'there'
        stat["R1"]['cDBU2-primer'] = 'absent'
        stat["R1"]['UPTAG-frag']   = 'absent'
        stat["R1"]['lox2272']      = 'absent'

        stat["R2"]['DBD2-primer']  = 'there'
        stat["R2"]['cDBD1-primer'] = 'absent'
        stat["R2"]['cDNTAG-frag']  = 'absent'


        ##### Existence check; UPTAG
        if( 'cDBU2-primer' in R1.keys()):
            if( R1['DBU1-primer']['end_on_read'] < R1['cDBU2-primer']['str_on_read']):
                TAG_str = R1['DBU1-primer']['end_on_read']+1 -1
                TAG_end = R1['cDBU2-primer']['str_on_read']-1 -1

                stat["R1"]['cDBU2-primer'] = 'there'
                stat["R1"]['UPTAG-frag']   =  R1["seq"][TAG_str:TAG_end]



        ##### Existence check; lox2272
        if('cDBU2-primer'in R1.keys()):
            if('lox2272' in R1.keys()):
                if(R1['cDBU2-primer']['end_on_read'] < R1['lox2272']['str_on_read']):
                    offset = R1['lox2272']['str_on_read'] - R1['cDBU2-primer']['end_on_read'] -1
                    stat["R1"]['lox2272'] = 'there;offset=%d'%(offset)

        ##### Existence check; DNTAG
        if('cDBD1-primer' in R2.keys()):
            if(R2['DBD2-primer']['end_on_read'] < R2['cDBD1-primer']['str_on_read']):
                TAG_str = R2['DBD2-primer']['end_on_read'] 
                TAG_end = R2['cDBD1-primer']['str_on_read'] -1

                stat["R2"]['cDBD1-primer'] = 'there'
                stat["R2"]['cDNTAG-frag']  = R2["seq"][TAG_str:TAG_end]   


        ##### Sequence match check
        for read in stat:
            for primer in stat[read]:
                  if primer != "cDNTAG-frag":
                        if primer != "UPTAG-frag":
                            s =  stat[read][primer].split(";")
                            if s[0] == 'there':
                                if read =="R1":
                                    read_seq = R1['seq'][R1[primer]["str_on_read"]:R1[primer]["end_on_read"]] 
                                    if read_seq == seq_DB[primer]:
                                        stat[read][primer].replace("there","match")
                                    else:
                                        stat[read][primer].replace("there","no-match")
                                elif(read=="R2"):
                                    read_seq = R2['seq'][R2[primer]["str_on_read"]:R2[primer]["end_on_read"]] 
                                    if read_seq == seq_DB[primer]:
                                        stat[read][primer].replace("there","match")
                                    else:
                                        stat[read][primer].replace("there","no-match")
    
    elif (category =="DB-lox"):
        # Setting default status 
        stat["R1"]['DBloxP-primer']   = 'there'
        stat["R1"]['cloxP']           = 'absent'
        stat["R1"]['DBU1-primer']     = 'absent'
        stat["R1"]['UPTAG-frag']      = 'absent'                        
        stat["R1"]['cDBU2-primer']    = 'absent'

        stat["R2"]['DBlox2272-primer']= 'there'
        stat["R2"]['clox2272']        = 'absent'
        stat["R2"]['DBU2-primer']     = 'absent'
        stat["R2"]['cUPTAG-frag']     = 'absent'
        stat["R2"]['cDBU1-primer']    = 'absent'
        
        ##### Existence check; cLoxP
        if( 'cloxP' in R1.keys()):
            if( R1['DBloxP-primer']['end_on_read'] < R1['cloxP']['str_on_read']):
                offset = R1['cloxP']['str_on_read'] - R1['DBloxP-primer']['end_on_read'] -1
                stat["R1"]['cloxP'] = 'there;offset=%d'%(offset)

        #### Existence check; DBU1-primer
        if( 'cloxP' in R1.keys()):
            if( 'DBU1-primer' in R1.keys()):    
                if( R1['cloxP']['end_on_read'] < R1['DBU1-primer']['str_on_read']):
                    offset = R1['DBU1-primer']['str_on_read'] - R1['cloxP']['end_on_read'] -1
                    stat["R1"]['DBU1-primer'] = 'there;offset=%d'%(offset)

        #### Existence check; cDBU2-primer
        if( 'cDBU2-primer' in R1.keys()):
            if( 'DBU1-primer' in R1.keys()):    
                if( R1['DBU1-primer']['end_on_read'] < R1['cDBU2-primer']['str_on_read']):
                    offset = R1['cDBU2-primer']['str_on_read'] - R1['DBU1-primer']['end_on_read'] -1-25
                    stat["R1"]['cDBU2-primer'] = 'there;offset=%d'%(offset)

                    TAG_str = R1['DBU1-primer']['end_on_read']
                    TAG_end = R1['cDBU2-primer']['str_on_read']-1
                    stat["R1"]['UPTAG_frag'] = R1["seq"][TAG_str:TAG_end]

        ##### Existence check; clox2272
        if( 'clox2272' in R1.keys()):
            if( R1['cDBlox2272-primer']['end_on_read'] < R1['clox2272']['str_on_read']):
                offset = R1['clox2272']['str_on_read'] - R1['DBlox2272-primer']['end_on_read'] -1
                stat["R1"]['clox2272'] = 'there;offset=%d'%(offset)

        #### Existence check; DBU2-primer
        if( 'clox2272' in R2.keys()):
            if( 'DBU2-primer' in R2.keys()):    
                if( R2['clox2272']['end_on_read'] < R2['DBU2-primer']['str_on_read']):
                    offset = R2['DBU2-primer']['str_on_read'] - R2['clox2272']['end_on_read'] -1
                    stat["R2"]['DBU2-primer'] = 'there;offset=%d'%(offset)

        #### Existence check; cDBU1-primer
        if( 'DBU2-primer' in R2.keys()):
            if( 'cDBU1-primer' in R2.keys()):    
                if( R2['DBU2-primer']['end_on_read'] < R2['cDBU1-primer']['str_on_read']):
                    offset = R2['cDBU1-primer']['str_on_read'] - R2['DBU2-primer']['end_on_read'] -1-25
                    stat["R2"]['cDBU1-primer'] = 'there;offset=%d'%(offset)

                    TAG_str = R2['DBU2-primer']['end_on_read']
                    TAG_end = R2['cDBU1-primer']['str_on_read']-1
                    stat["R2"]['UPTAG_frag'] = R2["seq"][TAG_str:TAG_end]                       
                                

        ##### Sequence match check
        for read in stat:
            for primer in stat[read]:
                if primer != "cUPTAG-frag":
                        if primer != "UPTAG-frag":
                            s =  stat[read][primer].split(";")
                            if s[0] == 'there':
                                if read =="R1":
                                    read_seq = R1['seq'][R1[primer]["str_on_read"]:R1[primer]["end_on_read"]] 
                                    if read_seq == seq_DB[primer]:
                                        stat[read][primer].replace("there","match")
                                    else:
                                        stat[read][primer].replace("there","no-match")
                                elif(read=="R2"):
                                    read_seq = R2['seq'][R2[primer]["str_on_read"]:R2[primer]["end_on_read"]] 
                                    if read_seq == seq_DB[primer]:
                                        stat[read][primer].replace("there","match")
                                    else:
                                        stat[read][primer].replace("there","no-match")                

        #### Fragment match 
        #pprint(stat)
        if stat["R1"]["UPTAG-frag"] != "absent":
            if stat["R2"]["cUPTAG-frag"] != "absent":
                frag  = stat["R1"]["UPTAG-frag"]
                cfrag = stat["R2"]["cUPTAG-frag"]
                                
                BC    = BC_match(frag,cfrag)
                                
                stat['merged-UPTAG'] = BC   
            else:
                stat['merged-UPTAG'] = "absent"
        else:
            stat['merged-UPTAG'] = "absent"
    
    
                                
    elif (category == "AD-BC"):
        
        # Setting default status 
        stat["R1"]['ADU1-primer']  = 'there'
        stat["R1"]['cADU2-primer'] = 'absent'
        stat["R1"]['UPTAG-frag']   = 'absent'
        
        stat["R2"]['ADD2-primer']  = 'there'
        stat["R2"]['cADD1-primer'] = 'absent'
        stat["R2"]['cDNTAG-frag']  = 'absent'
        stat["R2"]['loxP']         = 'absent'
                                


        ##### Existence check; UPTAG
        if( 'cADU2-primer' in R1.keys()):
            if( R1['ADU1-primer']['end_on_read'] < R1['cADU2-primer']['str_on_read']):
                TAG_str = R1['ADU1-primer']['end_on_read']
                TAG_end = R1['cADU2-primer']['str_on_read']-1

                stat["R1"]['cADU2-primer'] = 'there'
                stat["R1"]['UPTAG-frag']        =  R1["seq"][TAG_str:TAG_end]



        ##### Existence check; loxP
        if('cADD1-primer'in R2.keys()):
            if('loxP' in  R2.keys()):
                if(R2['cADD1-primer']['end_on_read'] < R2['loxP']['str_on_read']):
                    offset = R2['loxP']['str_on_read'] - R2['cADD1-primer']['end_on_read'] -1
                    stat["R2"]['loxP'] = 'there;offset=%d'%(offset)

        ##### Existence check; DNTAG
        if('cADD1-primer' in R2.keys()):
            if(R2['ADD2-primer']['end_on_read'] < R2['cADD1-primer']['str_on_read']):
                TAG_str = R2['ADD2-primer']['end_on_read']
                TAG_end = R2['cADD1-primer']['str_on_read']-1

                stat["R2"]['cADD1-primer'] = 'there'
                stat["R2"]['cDNTAG-frag']  = R2["seq"][TAG_str:TAG_end]   


        ##### Sequence match check
        #print(stat)
        for read in stat:
            for primer in stat[read]:
                if primer != "cDNTAG-frag":
                    if primer != "UPTAG-frag":
                        s =  stat[read][primer].split(";")
                        if s[0] == 'there':
                            if read =="R1":
                                read_seq = R1['seq'][R1[primer]["str_on_read"]:R1[primer]["end_on_read"]] 
                                if read_seq == seq_DB[primer]:
                                    stat[read][primer].replace("there","match")
                                else:
                                    stat[read][primer].replace("there","no-match")
                            elif(read=="R2"):
                                read_seq = R2['seq'][R2[primer]["str_on_read"]:R2[primer]["end_on_read"]] 
                                if read_seq == seq_DB[primer]:
                                    stat[read][primer].replace("there","match")
                                else:
                                    stat[read][primer].replace("there","no-match")

    elif (category =="AD-lox"):
        #pprint(R1)
        #pprint(R2)
        # Setting default status 
        stat["R1"]['ADloxP-primer']   = 'there'
        stat["R1"]['cloxP']           = 'absent'
        stat["R1"]['ADD1-primer']     = 'absent'
        stat["R1"]['DNTAG-frag']      = 'absent'                        
        stat["R1"]['cADD2-primer']    = 'absent'

        stat["R2"]['ADlox2272-primer']= 'there'
        stat["R2"]['clox2272']        = 'absent'
        stat["R2"]['ADD2-primer']= 'absent'
        stat["R2"]['cDNTAG-frag']= 'absent'
        stat["R2"]['cADD1-primer']= 'absent'
        #pprint(stat)
        ##### Existence check; cLoxP
        if( 'cloxP' in R1.keys()):
            if( R1['ADloxP-primer']['end_on_read'] < R1['cloxP']['str_on_read']):
                offset = R1['cloxP']['str_on_read'] - R1['ADloxP-primer']['end_on_read'] -1
                stat["R1"]['cloxP'] = 'there;offset=%d'%(offset)
                #print('cloxP')
        #### Existence check; ADD1-primer
        if( 'cloxP' in R1.keys()):
            if( 'ADD1-primer' in R1.keys()):    
                if( R1['cloxP']['end_on_read'] < R1['ADD1-primer']['str_on_read']):
                    offset = R1['ADD1-primer']['str_on_read'] - R1['cloxP']['end_on_read'] -1
                    stat["R1"]['ADD1-primer'] = 'there;offset=%d'%(offset)
                    #print('ADD1-primer')

        #### Existence check; cADD2-primer
        if( 'cADD2-primer' in R1.keys()):
            if( 'ADD1-primer' in R1.keys()):    
                if( R1['ADD1-primer']['end_on_read'] < R1['cADD2-primer']['str_on_read']):
                    offset = R1['cADD2-primer']['str_on_read'] - R1['ADD1-primer']['end_on_read'] -1-25
                    stat["R1"]['cADD2-primer'] = 'there;offset=%d'%(offset)

                    TAG_str = R1['ADD1-primer']['end_on_read']
                    TAG_end = R1['cADD2-primer']['str_on_read']-1
                    stat["R1"]['DNTAG-frag'] = R1["seq"][TAG_str:TAG_end]
                    #print('cADD2-primer')

        ##### Existence check; clox2272
        if( 'clox2272' in R2.keys()):
            if ('ADlox2272-primer' in R2.keys()):
                if( R2['ADlox2272-primer']['end_on_read'] < R2['clox2272']['str_on_read']):
                    offset = R2['clox2272']['str_on_read'] - R2['ADlox2272-primer']['end_on_read'] -1
                    stat["R2"]['clox2272'] = 'there;offset=%d'%(offset)
                    #print('lox2272')
        #### Existence check; ADD2-primer
        if( 'clox2272' in R2.keys()):
            if( 'ADD2-primer' in R2.keys()):    
                if( R2['clox2272']['end_on_read'] < R2['ADD2-primer']['str_on_read']):
                    offset = R2['ADD2-primer']['str_on_read'] - R2['clox2272']['end_on_read'] -1
                    stat["R2"]['ADD2-primer'] = 'there;offset=%d'%(offset)
                    #print('ADD2-primer')
        #### Existence check; cADD1-primer
        if( 'ADD2-primer' in R2.keys()):
            if( 'cADD1-primer' in R2.keys()):    
                if( R2['ADD2-primer']['end_on_read'] < R2['cADD1-primer']['str_on_read']):
                    offset = R2['cADD1-primer']['str_on_read'] - R2['ADD2-primer']['end_on_read'] -1-25
                    stat["R2"]['cADD1-primer'] = 'there;offset=%d'%(offset)
                    #print('cADD1-primer')
                    TAG_str = R2['ADD2-primer']['end_on_read'] 
                    TAG_end = R2['cADD1-primer']['str_on_read']-1
                    stat["R2"]['cDNTAG-frag'] = R2["seq"][TAG_str:TAG_end]                       
                    #print('cDNTAG-frag')
        #pprint(R1)
        #pprint(R2)
        #pprint(stat)
        ##### Sequence match check
        for read in stat:
            for primer in stat[read]:
                if primer != "cDNTAG-frag":
                    if primer != "DNTAG-frag":
                        s =  stat[read][primer].split(";")
                        if s[0] == 'there':
                            if read =="R1":
                                read_seq = R1['seq'][R1[primer]["str_on_read"]:R1[primer]["end_on_read"]] 
                                if read_seq == seq_DB[primer]:
                                    stat[read][primer].replace("there","match")
                                else:
                                    stat[read][primer].replace("there","no-match")
                            elif(read=="R2"):
                                read_seq = R2['seq'][R2[primer]["str_on_read"]:R2[primer]["end_on_read"]] 
                                if read_seq == seq_DB[primer]:
                                    stat[read][primer].replace("there","match")
                                else:
                                    stat[read][primer].replace("there","no-match")
                        
        #### Fragment match 
        #pprint(stat)
        if stat["R1"]["DNTAG-frag"] != "absent":
            if stat["R2"]["cDNTAG-frag"] != "absent":
                frag  = stat["R1"]["DNTAG-frag"]
                cfrag = stat["R2"]["cDNTAG-frag"]
                                
                BC    = BC_match(frag,cfrag)
                                
                stat['merged-DNTAG'] = BC    
            else:
                stat['merged-DNTAG'] = "absent"
        else:
            stat['merged-DNTAG'] = "absent"
    R1 = {}
    R2 = {}
    return stat

In [17]:
data = {}
seq_DB   = fasta2dict("%s/const-seq.fna"%(PATH))
f_pairs = []
for f in range(0, len(fasta_files)):
    for f2 in range(0, len(fasta_files)):
        if "R1" in fasta_files[f]:
            if fasta_files[f].replace("R1","R2") == fasta_files[f2]:
                f_pairs.append([fasta_files[f].split(".")[0].split("_")[-1],fasta_files[f],fasta_files[f2]])
f_pairs.sort(key=lambda x: x[0])
#pprint(f_pairs)
print ("========== Parsing BLAST output ============")
num = 1
for small_file in f_pairs:
    print ("Now working on subfiles : %s & %s "%(small_file[1],small_file[2]))
    R1_ID = small_file[1].split(".")[0]
    R2_ID = small_file[2].split(".")[0]

    R1_fna = fasta2dict('%s/Data/%s/fragmented_fasta/%s'%(PATH,dir_name,small_file[1]))
    R2_fna = fasta2dict('%s/Data/%s/fragmented_fasta/%s'%(PATH,dir_name,small_file[2]))

    R1_blast = '%s/Data/%s/blast/out.blast/%s.blast'%(PATH,dir_name,R1_ID)
    R2_blast = '%s/Data/%s/blast/out.blast/%s.blast'%(PATH,dir_name,R2_ID)
    
    #print(R1_ID)
    #print(R2_ID)

    d = parse_blast_output2dict(R1_blast,R2_blast,PATH,dir_name,tag_f)
       
    for read in tqdm(d.keys()):
        target = 0
        try:
            d[read]["R1"]["seq"] = R1_fna[read]
            d[read]["R2"]["seq"] = R2_fna[read]

            target,category,P_SEQ1,P_SEQ2,R_SEQ,C_SEQ = assign_category(d[read])
            
        except KeyError as e:
            #print( e)
            pass
        if target ==1:
            #print ("Read: %s \n Category:%s\n Plate tag 1: %s\n Plate tag 2: %s\n Row tag: %s\n Col tag: %s\n\n\n\n"%(read,category,P_SEQ1,P_SEQ2,R_SEQ,C_SEQ))

            P_TAG1 = "P%02d" %(int(tag_calling(bar_d,P_SEQ1)))
            P_TAG2 = "P%02d" %(int(tag_calling(bar_d,P_SEQ2)))
            P_TAGs = "%s-%s"%(P_TAG1,P_TAG2)
            R_TAG  = "R%02d" %(int(tag_calling(bar_d,R_SEQ)))
            C_TAG  = "C%02d" %(int(tag_calling(bar_d,C_SEQ)))

            if int(tag_calling(bar_d,P_SEQ1))*int(tag_calling(bar_d,P_SEQ2))*int(tag_calling(bar_d,R_SEQ))*int(tag_calling(bar_d,C_SEQ))!=0:
                #pprint("%s\n%s\n%s\n%s\n%s"%(read,category,P_TAGs,R_TAG,C_TAG))
                #pprint(d[read]['R1'])
                #pprint(d[read]['R2'])
                stat = analyze_stat(category,d[read]['R1'],d[read]['R2'],seq_DB) 
                #d = remove_key(d,read)
            
                #pprint(stat)
                try:
                    data[category][P_TAGs][R_TAG][C_TAG][read] = stat
                except KeyError:
                    try:
                        data[category][P_TAGs][R_TAG][C_TAG]       = {}
                        data[category][P_TAGs][R_TAG][C_TAG][read] = stat
                    except KeyError:
                        try:
                            data[category][P_TAGs][R_TAG]              = {}
                            data[category][P_TAGs][R_TAG][C_TAG]       = {}
                            data[category][P_TAGs][R_TAG][C_TAG][read] = stat
                        except KeyError:
                            try:
                                data[category][P_TAGs]                     = {}
                                data[category][P_TAGs][R_TAG]              = {}
                                data[category][P_TAGs][R_TAG][C_TAG]       = {}
                                data[category][P_TAGs][R_TAG][C_TAG][read] = stat
                            except KeyError:
                                data[category]                             = {}
                                data[category][P_TAGs]                     = {}
                                data[category][P_TAGs][R_TAG]              = {}
                                data[category][P_TAGs][R_TAG][C_TAG]       = {}
                                data[category][P_TAGs][R_TAG][C_TAG][read] = stat




    #pprint(data)
    print("==========         DONE         ============\n\nCreating : %s/Data/%s/identification/parsed_blast_data_%d.pickle"%(PATH,dir_name,num))
    with open("%s/Data/%s/identification/parsed_blast_data_%d.pickle"%(PATH,dir_name,num),'wb') as f:
        pickle.dump(data,f)
    print("Done")
    f.close()
    num+=1

========== Parsing BLAST output ============
Now working on subfiles : test_R1_1.fna & test_R2_1.fna 


100%|██████████| 77214/77214 [00:48<00:00, 1605.73it/s]

==========         DONE         ============

Creating : /Users/danyamamotoevans/GitHub/BFG-Y2H/RCP-PCR/Data/test/identification/parsed_blast_data_1.pickle
Done


In [18]:
#This is a pause point.
#The following cell can be run without any previous 'heavy' cells.


#Glimps at the data
with open("%s/Data/%s/identification/parsed_blast_data_1.pickle"%(PATH,dir_name), 'rb') as F:
        da = pickle.load(F)
#pprint(da)
        
typs = ["DB-BC","AD-BC","AD-lox"]
for i in typs:
    co = 0
    for j in da[i]:
        for k in da[i][j]:
            for c in da[i][j][k]:
                co +=1
                if co < 2:
                    print("%s\t%s\t%s\t%s"%(i,j,k,c))
                    pprint(da[i][j][k][c])

DB-BC	P04-P09	R14	C02
{'M00244_186_000000000-BFK5B_1_1101_16364_1361': {'R1': {'DBU1-primer': 'there',
                                                         'UPTAG-frag': 'GGGGTTCTGCCGTAGGGTATCTGT',
                                                         'cDBU2-primer': 'there',
                                                         'lox2272': 'there;offset=0'},
                                                  'R2': {'DBD2-primer': 'there',
                                                         'cDBD1-primer': 'there',
                                                         'cDNTAG-frag': 'GCTAGGCCTGAGTTCCTGGATTGGA'}},
 'M00244_186_000000000-BFK5B_1_1101_18060_4273': {'R1': {'DBU1-primer': 'there',
                                                         'UPTAG-frag': 'GTTCCTTCTGTGCAGGACTTCTTT',
                                                         'cDBU2-primer': 'there',
                                                         'lox2272': 'there;offset=0'},
              

In [19]:
# For each of the parsed BLAST output, re-organize the data by barcode counts per de-multiplexed sample

def get_pickle(x):
    files = []
    lis = os.listdir("%s" %x)
    for i in lis:
        if i[-7:] == ".pickle" :
            files.append(i)
    files.sort()
    return files


list_pickle_files = get_pickle("%s/Data/%s/identification"%(PATH,dir_name))

count_data = {}

for f in list_pickle_files:
    with open("%s/Data/%s/identification/%s"%(PATH,dir_name,f), 'rb') as F:
        data_in_small_chunk = pickle.load(F)

        for rcp in data_in_small_chunk:
            for P_TAG in data_in_small_chunk[rcp]:
                for R_TAG in data_in_small_chunk[rcp][P_TAG]:
                    for C_TAG in data_in_small_chunk[rcp][P_TAG][R_TAG]:
                        for read in data_in_small_chunk[rcp][P_TAG][R_TAG][C_TAG]:       
                            el_d = {}
                            for R in ["R1","R2"]:
                                for el in data_in_small_chunk[rcp][P_TAG][R_TAG][C_TAG][read][R]:
                                    el_d[el] = data_in_small_chunk[rcp][P_TAG][R_TAG][C_TAG][read][R][el]
                            #print(rcp)
                            #pprint(el_d)
                            
                            if rcp == "DB-BC":
                                ####################
                                ## DB-BC
                                ####################
                                UPTAG  = el_d["UPTAG-frag"] 
                                cDNTAG = el_d["cDNTAG-frag"]
                                if cDNTAG != 'absent':
                                    DNTAG  = rv_comp(cDNTAG)
                                else:
                                    DNTAG  = cDNTAG
                                BC_pair= "%s:%s"%(UPTAG,DNTAG) 
                              
                                els    = ['DBU1-primer','cDBU2-primer','cDBD1-primer','DBD2-primer']                                
                                status = (":").join([el_d[el] for el in els])                 
                                try:
                                    count_data[tags[P_TAG][0]][R_TAG][C_TAG][rcp][BC_pair][status] += 1
                                except KeyError:
                                    try:
                                        count_data[tags[P_TAG][0]][R_TAG][C_TAG][rcp][BC_pair] = {}
                                        count_data[tags[P_TAG][0]][R_TAG][C_TAG][rcp][BC_pair][status] = 1
                                    except KeyError:
                                        try:
                                            count_data[tags[P_TAG][0]][R_TAG][C_TAG][rcp] = {}
                                            count_data[tags[P_TAG][0]][R_TAG][C_TAG][rcp][BC_pair] = {}
                                            count_data[tags[P_TAG][0]][R_TAG][C_TAG][rcp][BC_pair][status] = 1
                                        except KeyError:
                                            try:
                                                count_data[tags[P_TAG][0]][R_TAG][C_TAG] = {}
                                                count_data[tags[P_TAG][0]][R_TAG][C_TAG][rcp] = {}
                                                count_data[tags[P_TAG][0]][R_TAG][C_TAG][rcp][BC_pair] = {}
                                                count_data[tags[P_TAG][0]][R_TAG][C_TAG][rcp][BC_pair][status] = 1

                                            except KeyError:
                                                try:
                                                    count_data[tags[P_TAG][0]][R_TAG] = {}
                                                    count_data[tags[P_TAG][0]][R_TAG][C_TAG] = {}
                                                    count_data[tags[P_TAG][0]][R_TAG][C_TAG][rcp] = {}
                                                    count_data[tags[P_TAG][0]][R_TAG][C_TAG][rcp][BC_pair] = {}
                                                    count_data[tags[P_TAG][0]][R_TAG][C_TAG][rcp][BC_pair][status] = 1


                                                except KeyError:
                                                    try:
                                                        count_data[tags[P_TAG][0]] = {}
                                                        count_data[tags[P_TAG][0]][R_TAG] = {}
                                                        count_data[tags[P_TAG][0]][R_TAG][C_TAG] = {}
                                                        count_data[tags[P_TAG][0]][R_TAG][C_TAG][rcp] = {}
                                                        count_data[tags[P_TAG][0]][R_TAG][C_TAG][rcp][BC_pair] = {}
                                                        count_data[tags[P_TAG][0]][R_TAG][C_TAG][rcp][BC_pair][status] = 1

                                                    except KeyError:
                                                        pass
                            if rcp == "DB-lox":
                                ####################
                                ## DB-lox
                                ####################
                                merged_UPTAG  = data_in_small_chunk[rcp][P_TAG][R_TAG][C_TAG][read]["merged-UPTAG"]
                                
                                els    = ['cloxP','clox2272','DBU1-primer','DBU2-primer']                                
                                status = (":").join([el_d[el] for el in els])                 
                                try:
                                    count_data[tags[P_TAG][0]][R_TAG][C_TAG][rcp][merged_UPTAG][status] += 1
                                except KeyError:
                                    try:
                                        count_data[tags[P_TAG][0]][R_TAG][C_TAG][rcp][merged_UPTAG] = {}
                                        count_data[tags[P_TAG][0]][R_TAG][C_TAG][rcp][merged_UPTAG][status] = 1
                                    except KeyError:
                                        try:
                                            count_data[tags[P_TAG][0]][R_TAG][C_TAG][rcp] = {}
                                            count_data[tags[P_TAG][0]][R_TAG][C_TAG][rcp][merged_UPTAG] = {}
                                            count_data[tags[P_TAG][0]][R_TAG][C_TAG][rcp][merged_UPTAG][status] = 1
                                        except KeyError:
                                            try:
                                                count_data[tags[P_TAG][0]][R_TAG][C_TAG] = {}
                                                count_data[tags[P_TAG][0]][R_TAG][C_TAG][rcp] = {}
                                                count_data[tags[P_TAG][0]][R_TAG][C_TAG][rcp][merged_UPTAG] = {}
                                                count_data[tags[P_TAG][0]][R_TAG][C_TAG][rcp][merged_UPTAG][status] = 1

                                            except KeyError:
                                                try:
                                                    count_data[tags[P_TAG][0]][R_TAG] = {}
                                                    count_data[tags[P_TAG][0]][R_TAG][C_TAG] = {}
                                                    count_data[tags[P_TAG][0]][R_TAG][C_TAG][rcp] = {}
                                                    count_data[tags[P_TAG][0]][R_TAG][C_TAG][rcp][merged_UPTAG] = {}
                                                    count_data[tags[P_TAG][0]][R_TAG][C_TAG][rcp][merged_UPTAG][status] = 1


                                                except KeyError:
                                                    try:
                                                        count_data[tags[P_TAG][0]] = {}
                                                        count_data[tags[P_TAG][0]][R_TAG] = {}
                                                        count_data[tags[P_TAG][0]][R_TAG][C_TAG] = {}
                                                        count_data[tags[P_TAG][0]][R_TAG][C_TAG][rcp] = {}
                                                        count_data[tags[P_TAG][0]][R_TAG][C_TAG][rcp][merged_UPTAG] = {}
                                                        count_data[tags[P_TAG][0]][R_TAG][C_TAG][rcp][merged_UPTAG][status] = 1

                                                    except KeyError:
                                                        pass
                            if rcp == "AD-BC":
                                ####################
                                ## AD-BC
                                ####################
                                UPTAG  = el_d["UPTAG-frag"] 
                                cDNTAG = el_d["cDNTAG-frag"]
                                if cDNTAG != 'absent':
                                    DNTAG  = rv_comp(cDNTAG)
                                else:
                                    DNTAG  = cDNTAG
                                BC_pair= "%s:%s"%(UPTAG,DNTAG) 

                                els    = ['ADU1-primer','cADU2-primer','cADD1-primer','ADD2-primer']                                
                                status = (":").join([el_d[el] for el in els])                 
                                try:
                                    count_data[tags[P_TAG][0]][R_TAG][C_TAG][rcp][BC_pair][status] += 1
                                except KeyError:
                                    try:
                                        count_data[tags[P_TAG][0]][R_TAG][C_TAG][rcp][BC_pair] = {}
                                        count_data[tags[P_TAG][0]][R_TAG][C_TAG][rcp][BC_pair][status] = 1
                                    except KeyError:
                                        try:
                                            count_data[tags[P_TAG][0]][R_TAG][C_TAG][rcp] = {}
                                            count_data[tags[P_TAG][0]][R_TAG][C_TAG][rcp][BC_pair] = {}
                                            count_data[tags[P_TAG][0]][R_TAG][C_TAG][rcp][BC_pair][status] = 1
                                        except KeyError:
                                            try:
                                                count_data[tags[P_TAG][0]][R_TAG][C_TAG] = {}
                                                count_data[tags[P_TAG][0]][R_TAG][C_TAG][rcp] = {}
                                                count_data[tags[P_TAG][0]][R_TAG][C_TAG][rcp][BC_pair] = {}
                                                count_data[tags[P_TAG][0]][R_TAG][C_TAG][rcp][BC_pair][status] = 1

                                            except KeyError:
                                                try:
                                                    count_data[tags[P_TAG][0]][R_TAG] = {}
                                                    count_data[tags[P_TAG][0]][R_TAG][C_TAG] = {}
                                                    count_data[tags[P_TAG][0]][R_TAG][C_TAG][rcp] = {}
                                                    count_data[tags[P_TAG][0]][R_TAG][C_TAG][rcp][BC_pair] = {}
                                                    count_data[tags[P_TAG][0]][R_TAG][C_TAG][rcp][BC_pair][status] = 1


                                                except KeyError:
                                                    try:
                                                        count_data[tags[P_TAG][0]] = {}
                                                        count_data[tags[P_TAG][0]][R_TAG] = {}
                                                        count_data[tags[P_TAG][0]][R_TAG][C_TAG] = {}
                                                        count_data[tags[P_TAG][0]][R_TAG][C_TAG][rcp] = {}
                                                        count_data[tags[P_TAG][0]][R_TAG][C_TAG][rcp][BC_pair] = {}
                                                        count_data[tags[P_TAG][0]][R_TAG][C_TAG][rcp][BC_pair][status] = 1

                                                    except KeyError:
                                                        pass
                            if rcp == "AD-lox":
                                ####################
                                ## AD-lox
                                ####################
                                #pprint(data_in_small_chunk[rcp][P_TAG][R_TAG][C_TAG][read])
                                try:
                                    merged_DNTAG  = data_in_small_chunk[rcp][P_TAG][R_TAG][C_TAG][read]["merged-DNTAG"]



                                    els    = ['cloxP','clox2272','ADD1-primer','ADD2-primer']                                
                                    status = (":").join([el_d[el] for el in els])                 
                                    try:
                                        count_data[tags[P_TAG][0]][R_TAG][C_TAG][rcp][merged_DNTAG][status] += 1
                                    except KeyError:
                                        try:
                                            count_data[tags[P_TAG][0]][R_TAG][C_TAG][rcp][merged_DNTAG] = {}
                                            count_data[tags[P_TAG][0]][R_TAG][C_TAG][rcp][merged_DNTAG][status] = 1
                                        except KeyError:
                                            try:
                                                count_data[tags[P_TAG][0]][R_TAG][C_TAG][rcp] = {}
                                                count_data[tags[P_TAG][0]][R_TAG][C_TAG][rcp][merged_DNTAG] = {}
                                                count_data[tags[P_TAG][0]][R_TAG][C_TAG][rcp][merged_DNTAG][status] = 1
                                            except KeyError:
                                                try:
                                                    count_data[tags[P_TAG][0]][R_TAG][C_TAG] = {}
                                                    count_data[tags[P_TAG][0]][R_TAG][C_TAG][rcp] = {}
                                                    count_data[tags[P_TAG][0]][R_TAG][C_TAG][rcp][merged_DNTAG] = {}
                                                    count_data[tags[P_TAG][0]][R_TAG][C_TAG][rcp][merged_DNTAG][status] = 1

                                                except KeyError:
                                                    try:
                                                        count_data[tags[P_TAG][0]][R_TAG] = {}
                                                        count_data[tags[P_TAG][0]][R_TAG][C_TAG] = {}
                                                        count_data[tags[P_TAG][0]][R_TAG][C_TAG][rcp] = {}
                                                        count_data[tags[P_TAG][0]][R_TAG][C_TAG][rcp][merged_DNTAG] = {}
                                                        count_data[tags[P_TAG][0]][R_TAG][C_TAG][rcp][merged_DNTAG][status] = 1


                                                    except KeyError:
                                                        try:
                                                            count_data[tags[P_TAG][0]] = {}
                                                            count_data[tags[P_TAG][0]][R_TAG] = {}
                                                            count_data[tags[P_TAG][0]][R_TAG][C_TAG] = {}
                                                            count_data[tags[P_TAG][0]][R_TAG][C_TAG][rcp] = {}
                                                            count_data[tags[P_TAG][0]][R_TAG][C_TAG][rcp][merged_DNTAG] = {}
                                                            count_data[tags[P_TAG][0]][R_TAG][C_TAG][rcp][merged_DNTAG][status] = 1

                                                        except KeyError:
                                                            pass
                                except KeyError:
                                    pass
                                    #print (data_in_small_chunk[rcp][P_TAG][R_TAG][C_TAG][read]["R1"]["DNTAG-frag"])



        F.close()
#pprint(count_data)
        
with open("%s/Data/%s/count/merged_count_data.pickle"%(PATH,dir_name),'wb') as f:
    pickle.dump(count_data,f)
f.close()

        


In [20]:
#Glimps at the data
with open("%s/Data/%s/count/merged_count_data.pickle"%(PATH,dir_name), 'rb') as F:
        da = pickle.load(F)
co = 0
for i in da:
    for j in da[i]:
        for k in da[i][j]:
            for c in da[i][j][k]:
                if co < 10:
                    print("%s\t%s\t%s\t%s"%(i,j,k,c))
                    pprint(da[i][j][k][c])
                    co +=1

AD006	R15	C10	AD-BC
{'CATGCTGTTGTTTCGGTCGTGTGCC:CACAGACTCCCACCGTCCATCCTCC': {'there:there:there:there': 1},
 'TTCGCTTGTATTCCGGACGTTGCTT:ACAATCCTAACGTTAAATAGCGCAA': {'there:there:there:there': 1}}
AD006	R15	C10	AD-lox
{'CAGACTAAACTGGGACTATCAGACT': {'there;offset=0:there;offset=0:there;offset=0:there;offset=0': 6},
 'GGGTAAAGGACCGAGCAAACCGTGG': {'there;offset=0:there;offset=0:there;offset=0:there;offset=0': 1}}
AD006	R15	C06	AD-BC
{'TCTGCCTGTTTGGTTCTTTTGTCCG:TTCAAACGTCAACTACCCGACATAG': {'there:there:there:there': 4}}
AD006	R15	C06	AD-lox
{'ATAACAATAAAGAACAACACAGCGA': {'there;offset=0:there;offset=0:there;offset=0:there;offset=0': 1},
 'CTCTCACCACCTTACACCCACCTTT': {'there;offset=0:there;offset=0:there;offset=0:there;offset=0': 1},
 'TTCAAACGTCAACTACCCGACATAG': {'there;offset=0:there;offset=0:there;offset=0:there;offset=0': 4}}
AD006	R15	C21	AD-BC
{'ACGATTCACCTTTCCGTTGCTGGCG:ATAACAATAAAGAACAACACAGCGA': {'there:there:there:there': 1},
 'CATGCTGTTGTTTCGGTCGTGTGCC:ATAACAATAAAGAACAACACAGCGA': 

In [26]:
#################################################
# Functions to call barcode
################################################

def evaluate(element):

    primer_order = ['U1','U2','D1','D2']
    lox_order    = ['loxP','lox2272']

    if(float(element["BC"]['min_BC_hit_rate_impt']) > 0.1): ##### THINK ABOUT THIS THRESHOLD
        try:
            primer_array = element["BC"]['likely_stat'].split(":") 
        except KeyError:
            primer_array = ["N_A","N_A","N_A","N_A"]
            
        try:
            lox_array    = element["lox"]['likely_stat'].split(":") 
        except KeyEror:
            lox_array = ["N_A","N_A"]
        
    primer_stat = {}
    for i in range(0,4):
        if(float(element["BC"]['stat_rate']) > 0.5):
            if primer_array[i] == "match": #may not be perfect match. Check 
                primer_stat[primer_order[i]] = '+'
        elif(float(element["BC"]['stat_rate']) > 0.5 ):
            if primer_array[i] != "match":
                primer_stat[primer_order[i]] = '-'
        else:
            primer_stat[primer_order[i]] = 'Ab'

    lox_stat = {}
    for i in range(0,2):
        if(float(element["lox"]['stat_rate']) > 0.5):
            if lox_array[i] == "match": #may not be perfect match. Check 
                lox_stat[lox_order[i]] = '+'
        elif(float(element["lox"]['stat_rate']) > 0.5 ):
            if lox_array[i] != "match":
                lox_stat[lox_order[i]] = '-'
        else:
            lox_stat[lox_order[i]] = 'Ab'


    evaluated = element
    evaluated["BC"]['likely_stat']  = primer_stat
    evaluated["lox"]['likely_stat'] = lox_stat

    return evaluated

def match(barcode_i,barcode_j):
    match_d = {}
    barcode_i_nuc = barcode_i.replace(":","")
    barcode_j_nuc = barcode_j.replace(":","")
    min_len = min(len(barcode_i_nuc),len(barcode_j_nuc))

    num = 0
    for n in range(min_len):
        ni = barcode_i_nuc[n]
        nj = barcode_j_nuc[n]
        if (ni ==nj):
            num+=1

            
    match_d['num'] = num
    match_d['rate']= float(num)/min_len
    
    return match_d

def ErrorProp_div(Hit,All):
    val = 0
    error = 0

    Hit = float(Hit)
    All = float(All)


    val = Hit/All
    d_All = All **(0.5)
    #print All,d_All
    d_Hit = Hit **(0.5)
    #print target, d_target
    error = ((d_All/All) ** 2) + ((d_Hit/Hit) ** 2)
    error **= 0.5
    error  *= val

    return (val,error)


def cal_rate(count_all):
    BC_all  = count_all['BC_all']
    lox_all = count_all['lox_all']

    data = count_all["Data"]
    #pprint(count_all)
    
   
    
    for barcode in data:
        #print(barcode)
        BC_hit_rate, BC_hit_rate_error = ErrorProp_div(data[barcode]['BC']['hit'], BC_all)
        min_BC_hit_rate   = BC_hit_rate   - BC_hit_rate_error

        data[barcode]['BC']['min_BC_hit_rate_impt'] = "%.4f"%(min_BC_hit_rate)

        BC_stat_rate, BC_stat_rate_error  = ErrorProp_div(data[barcode]['BC']['likely_stat_count'],data[barcode]['BC']['hit'])
        try:
            lox_stat_rate,lox_stat_rate_error = ErrorProp_div(data[barcode]['lox']['likely_stat_count'],data[barcode]['lox']['hit'])
        except ZeroDivisionError:
            lox_stat_rate = 0
            lox_stat_rate_error = 0
        

        min_BC_stat_rate  = "%.4f"%float(BC_stat_rate  - BC_stat_rate_error)
        min_lox_stat_rate = "%.4f"%float(lox_stat_rate - lox_stat_rate_error)
        data[barcode]['BC']['stat_rate']  = "%.4f"%float(min_BC_stat_rate)
        data[barcode]['lox']['stat_rate'] = "%.4f"%float(min_lox_stat_rate)

        count_all['Data'] = data
  
    return count_all




def call_barcode(data,ad_db):
    count_all = {"BC_all":0,"lox_all":0,"Data":{}}
    BC_all  = 0
    lox_all = 0
        
    pas = 0
    ####################################
    # Count useful BC tag and its stat #
    ####################################
    pattern = '^[ATGCN]+\:[ATGCN]+'
    count_array = []
    #pprint(data)
    for barcode in data["%s-BC"%(ad_db)]:
        if re.match(pattern,barcode):
            counts      = {}
            counts['barcode']         = barcode
            pas = 1
            element                 = data["%s-BC"%(ad_db)][barcode] 
            for stat in element:
                BC_all                    += element[stat]
                try:
                    
                    counts['count']['All']   += element[stat]    
                    counts['count'][stat]     = element[stat]
                except KeyError:
                    counts['count']           = {}
                    counts['count']['All']    = element[stat]    
                    counts['count'][stat]     = element[stat]

                        
                    
            count_array.append(counts)
        
    if pas ==1:
        #pprint(count_array)
        count_array = [[count['barcode'],count['count']['All'],count['count']] for count in count_array ]
        
        count_array.sort(key=lambda x: x[1],reverse=True)
        #pprint(count_array)
       


    #######################################
    # Count useful lox reads and its stat #
    #######################################
    pas2 = 0
    pattern = '^[ATGCN]+'
    for barcode in data["%s-lox"%(ad_db)]:
        if re.match(pattern,barcode):
            pas2 =1
            element                 = data["%s-lox"%(ad_db)][barcode] 
            for stat in element:
                    lox_all        += element[stat]
    
    count_all["BC_all"]  = BC_all
    count_all["lox_all"] = lox_all
    
   
    if (pas * pas2)==1:

        ########################
        ##### impute counts of similar barcodes 
        ########################
        if len(count_array) > 1:
            for i in range(len(count_array)):
                for j in range(1,len(count_array)-i):
                    J = j + i

                    barcode_i = count_array[i][0]
                    barcode_j = count_array[J][0]

                    all_i     = count_array[i][1]
                    all_j     = count_array[J][1]


                    if all_i > all_j:
                        match_d = match(barcode_i,barcode_j)
                        if match_d['rate'] > 0.8:
                            for stat in count_array[J][2]:
                                #print(stat)
                                try:
                                    count_array[i][2][stat] += count_array[J][2][stat]
                                except KeyError:
                                    count_array[i][2][stat] = count_array[J][2][stat] 
                                count_array[J][2][stat] = 0
                            count_array[J][1] = 0
                        

        ####################
        # Removing queries with no reads
        
        #pprint(count_array)            
        count_array.sort(key=lambda x: x[1],reverse=True)
        count_array = [i for i in count_array if i[1]>0]
        #pprint(count_array)            
        
        
        ##############################################
        ## Examining stat of Lox data for the same barcode
        
        
        #pprint(data["%s-lox"%(ad_db)])
        for bar in count_array:
            lox = {'count':{'All':0}}
            UPTAG = bar[0].split(":")[0]
            DNTAG = bar[0].split(":")[1]
            if ad_db =="DB":
                TAG = UPTAG
                lox['count'][TAG]= {'All':0}
            if ad_db =="AD":
                TAG = DNTAG
                lox['count'][TAG]= {'All':0}
        
            for lox_barcode in data["%s-lox"%(ad_db)]:
                pattern = '^[ATGCN]+'
                #pprint(lox_barcode)
                if re.match(pattern,lox_barcode):
                    if ad_db =="DB":
                        match_d = match(UPTAG,lox_barcode)
                    if ad_db =="AD":
                        match_d = match(DNTAG,lox_barcode)
                    
                    for stat in data["%s-lox"%(ad_db)][lox_barcode]:
                        #print(lox_barcode), print(stat),print(data["%s-lox"%(ad_db)][lox_barcode][stat])
                        lox['count']['All'] += data["%s-lox"%(ad_db)][lox_barcode][stat]
                        if match_d['rate'] > 0.8:
                            lox['count'][TAG]['All'] += data["%s-lox"%(ad_db)][lox_barcode][stat]
                            try:
                                lox['count'][TAG][stat]  += data["%s-lox"%(ad_db)][lox_barcode][stat]
                            except KeyError:
                                lox['count'][TAG][stat]   = data["%s-lox"%(ad_db)][lox_barcode][stat]
            #################
            ## Likely status of BC and Lox 

            BC_stat = [] 
            likely_BC_stat = 'absent'
            likely_BC_stat_count = 0
            for stat in bar[2]:  
                if (stat != 'All'):
                    BC_stat.append([stat,bar[2][stat]])   
                    BC_stat.sort(key=lambda x: x[1],reverse=True)
                    #pprint(BC_stat)

                    likely_BC_stat       = BC_stat[0][0]
                    likely_BC_stat_count += BC_stat[0][1]

            lox_stat = [] 
            #pprint(lox)
            likely_lox_stat       = 'absent'
            likely_lox_stat_count = 0

            for stat in lox['count'][TAG]:
                if (stat != 'All'):
                    lox_stat.append([stat,lox['count'][TAG][stat]])   

                    lox_stat.sort(key=lambda x: x[1],reverse=True)
                    #pprint(lox_stat)
                    likely_lox_stat       = lox_stat[0][0]
                    likely_lox_stat_count += lox_stat[0][1]


            #################################
            ## Data for screening good wells


            count_strct = {"BC":{},"lox":{}}

            count_strct["BC"]["UPTAG"]              = UPTAG
            count_strct["BC"]["DNTAG"]              = DNTAG
            count_strct["BC"]["hit"]                = bar[2]['All']
            count_strct["BC"]["likely_stat"]        = likely_BC_stat
            count_strct["BC"]["likely_stat_count"]  = likely_BC_stat_count
            count_strct["lox"]["hit"]               = lox['count'][TAG]['All']
            count_strct["lox"]["likely_stat"]       = likely_lox_stat
            count_strct["lox"]["likely_stat_count"] = likely_lox_stat_count

            count_all["Data"][bar[0]] = count_strct
        
        
        #pprint(count_all)

        rate_strct = cal_rate(count_all)

        return rate_strct




In [27]:
with open("%s/Data/%s/count/merged_count_data.pickle"%(PATH,dir_name), 'rb') as F:
    count_data = pickle.load(F)
    F.close()
#pprint(count_data)

evaluated = {}
count_data2 = call_barcode(count_data["AD003"]["R01"]["C04"],"AD")
pprint(count_data2)
for bc in count_data2['Data']:
    ev = evaluate(count_data2['Data'][bc])
"""
for plate in count_data:
    typ = plate[:2]
    for R in count_data[plate]:
        for C in count_data[plate][R]:
            #print(plate,R,C)
            #pprint(count_data[plate][R][C])
            if "%s-BC"%(typ) in count_data[plate][R][C]: 
                if "%s-lox"%(typ) in count_data[plate][R][C]:
                    count_data[plate][R][C] = call_barcode(count_data[plate][R][C],typ)
"""

{'BC_all': 9,
 'Data': {'ATCGTTTTTTGGTTTCAGTCCTGTA:CATTTAGGATAGAGCCACACTGAAA': {'BC': {'DNTAG': 'CATTTAGGATAGAGCCACACTGAAA',
                                                                         'UPTAG': 'ATCGTTTTTTGGTTTCAGTCCTGTA',
                                                                         'hit': 1,
                                                                         'likely_stat': 'there:there:there:there',
                                                                         'likely_stat_count': 1,
                                                                         'min_BC_hit_rate_impt': '-0.0060',
                                                                         'stat_rate': '-0.4142'},
                                                                  'lox': {'hit': 0,
                                                                          'likely_stat': 'absent',
                                                                          'likel

'\nfor plate in count_data:\n    typ = plate[:2]\n    for R in count_data[plate]:\n        for C in count_data[plate][R]:\n            #print(plate,R,C)\n            #pprint(count_data[plate][R][C])\n            if "%s-BC"%(typ) in count_data[plate][R][C]: \n                if "%s-lox"%(typ) in count_data[plate][R][C]:\n                    count_data[plate][R][C] = call_barcode(count_data[plate][R][C],typ)\n'

In [ ]:
#prepare test data from reads in MiSeq10-26-2017 out.identification files
# Vidualizing the output


#show
#from wand.image import Image as WImage
#img = WImage(filename='hat.pdf')
#img

In [ ]:




#Montecarlo simulation